<a href="https://colab.research.google.com/github/HuangJin-De/am_NH/blob/main/discussion/20231115.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! rm -rf am_NH
! git clone https://github.com/HuangJin-De/am_NH.git

Cloning into 'am_NH'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 185 (delta 76), reused 116 (delta 36), pack-reused 0
Receiving objects: 100% (185/185), 6.22 MiB | 25.27 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [12]:
import numpy as np
import matplotlib.pyplot as plt

path='am_NH'
filen=path+'/train_data/zm_21days_ori_data.dat'

data=np.fromfile(filen,dtype=np.float32)
data=data.reshape(-1,84)

z1=np.copy(data[:,0:21])
z2=np.copy(data[:,21:42])
m1=np.copy(data[:,42:63])
m2=np.copy(data[:,63:84])

print(z1.shape,z2.shape,m1.shape,m2.shape)


(4242, 21) (4242, 21) (4242, 21) (4242, 21)


estimations of b11, b12, b21, and b22.

In [31]:
from sklearn.linear_model import LinearRegression

def cal_reg_coef(x,y):
  model=LinearRegression()
  model.fit(x,y)
  return model.coef_


lag_z1z1=[]
lag_z1z2=[]
lag_z2z1=[]
lag_z2z2=[]
lag_m1z1=[]
lag_m1z2=[]

for i in np.arange(8,21):
  x=np.copy(z1[:,0]).reshape(-1,1)
  y=np.copy(z1[:,i]).reshape(-1,1)
  lag_z1z1.append(cal_reg_coef(x,y))

  x=np.copy(z1[:,0]).reshape(-1,1)
  y=np.copy(z2[:,i]).reshape(-1,1)
  lag_z2z1.append(cal_reg_coef(x,y))

  x=np.copy(z2[:,0]).reshape(-1,1)
  y=np.copy(z1[:,i]).reshape(-1,1)
  lag_z1z2.append(cal_reg_coef(x,y))

  x=np.copy(z2[:,0]).reshape(-1,1)
  y=np.copy(z2[:,i]).reshape(-1,1)
  lag_z2z2.append(cal_reg_coef(x,y))

  x=np.copy(z1[:,0]).reshape(-1,1)
  y=np.copy(m1[:,i]).reshape(-1,1)
  lag_m1z1.append(cal_reg_coef(x,y))

  x=np.copy(z2[:,0]).reshape(-1,1)
  y=np.copy(m1[:,i]).reshape(-1,1)
  lag_m1z2.append(cal_reg_coef(x,y))


#print(np.mean(lag_m1z1))
#print(np.mean(lag_m1z2))

a=np.matrix([[np.mean(lag_z1z1), np.mean(lag_z2z1)],
 [np.mean(lag_z1z2), np.mean(lag_z2z2)]])
b=np.matrix([[np.mean(lag_m1z1)], [np.mean(lag_m1z2)]])

print(a)
print(b)

x=np.matmul(np.linalg.inv(a),b)

print(x)


[[ 0.38850185 -0.07047598]
 [ 0.10965526  0.3179857 ]]
[[0.00036202]
 [0.00023971]]
[[0.00100567]
 [0.00040704]]
